In [1]:
import pandas as pd
import numpy as np

In [16]:
# Load both days
df1 = pd.read_csv("day1.csv")
df2 = pd.read_csv("day2.csv")

C:\Users\icarus\AppData\Local\Temp\ipykernel_33276\1521114905.py:1: DtypeWarning: Columns (86) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv("day1.csv")
C:\Users\icarus\AppData\Local\Temp\ipykernel_33276\1521114905.py:2: DtypeWarning: Columns (86) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv("day2.csv")


In [17]:
# Get counts
print(df1[" Label"].value_counts())
print(df2[" Label"].value_counts())

DrDoS_UDP        56866
DrDoS_DNS        56865
DrDoS_MSSQL      56865
Syn              56864
BENIGN           56863
DrDoS_SNMP       56863
TFTP             56863
UDP-lag          56863
DrDoS_LDAP       56862
DrDoS_SSDP       56862
DrDoS_NetBIOS    56861
DrDoS_NTP        56861
WebDDoS            439
Name:  Label, dtype: int64
Syn        113931
NetBIOS    113929
UDP        113928
MSSQL       81357
LDAP        66896
BENIGN      56965
Portmap     56965
UDPLag       1873
Name:  Label, dtype: int64


In [24]:
# Get benign samples from Day 2 and cull them to the lowest number of attack samples found in Day 1 (excluding webddos)
day2ben = df2[df2[" Label"] == "BENIGN"].copy()
day2ben = day2ben.sample(n=56861,random_state=42)

In [25]:
print(day2ben[" Label"].value_counts())

BENIGN    56861
Name:  Label, dtype: int64


In [21]:
# get list of labels so we can pull an equal number of samples from each.
cols = df1[" Label"].unique()
print(df1[" Label"].unique())

['BENIGN' 'DrDoS_DNS' 'DrDoS_LDAP' 'DrDoS_MSSQL' 'DrDoS_NetBIOS'
 'DrDoS_NTP' 'DrDoS_SNMP' 'DrDoS_SSDP' 'DrDoS_UDP' 'Syn' 'TFTP' 'WebDDoS'
 'UDP-lag']


In [26]:
# Assemble equal number of samples. Will be restructured in multiclass and binary datasets.
outMult = None

for val in cols:

    # too small, so we drop it
    if val == 'WebDDoS':
        continue

    # replace benign with day 2.
    elif val == "BENIGN":
        slice = day2ben

    # get all attacks of this type and cull to appropriate number
    else:
        slice = df1[df1[" Label"] == val].copy()
        slice = slice.sample(n=56861,random_state=42) # Ensure equal sample number

    # add to output
    if outMult is None:
        outMult = slice
    else:
        outMult = pd.concat([outMult, slice])

print(outMult[" Label"].value_counts())

BENIGN           56861
DrDoS_DNS        56861
DrDoS_LDAP       56861
DrDoS_MSSQL      56861
DrDoS_NetBIOS    56861
DrDoS_NTP        56861
DrDoS_SNMP       56861
DrDoS_SSDP       56861
DrDoS_UDP        56861
Syn              56861
TFTP             56861
UDP-lag          56861
Name:  Label, dtype: int64


In [28]:
# Drop the benign samples to get our multiclass dataset.
multiclass = outMult[outMult[" Label"] != "BENIGN"]

# Remove the annoying DrDOS prefix from labels.
multiclass[" Label"] = multiclass[" Label"].apply(lambda x: x.replace("DrDoS_", ""))

print(multiclass[" Label"].value_counts())

DNS        56861
LDAP       56861
MSSQL      56861
NetBIOS    56861
NTP        56861
SNMP       56861
SSDP       56861
UDP        56861
Syn        56861
TFTP       56861
UDP-lag    56861
Name:  Label, dtype: int64


C:\Users\icarus\AppData\Local\Temp\ipykernel_33276\2852405852.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  multiclass[" Label"] = multiclass[" Label"].apply(lambda x: x.replace("DrDoS_", ""))


In [29]:
# binary uses all columns
binary = outMult.copy()

# Remove the annoying DrDOS prefix from labels.
binary[" Label"] = binary[" Label"].apply(lambda x: x.replace("DrDoS_", ""))

print(binary[" Label"].value_counts())

BENIGN     56861
DNS        56861
LDAP       56861
MSSQL      56861
NetBIOS    56861
NTP        56861
SNMP       56861
SSDP       56861
UDP        56861
Syn        56861
TFTP       56861
UDP-lag    56861
Name:  Label, dtype: int64


In [31]:
# get attack labels
cols = binary[" Label"].unique()

# make new attack label comprised of equal subsets of attacks.
targ = int(56861 / 11)

outBin = None

for val in cols:

    # pull category
    slice = binary[binary[" Label"] == val].copy()

    # binary is equal to subset * 11 so it's equal.
    if val == "BENIGN":
        slice = slice.sample(n=targ * 11,random_state=42) # Ensure equal sample number

    # get subset and covert to attack type.
    else:
        slice = slice.sample(n=targ,random_state=42) # Ensure equal sample number
        slice[" Label"] = slice[" Label"].apply(lambda x: "ATTACK")

    # add to output
    if outBin is None:
        outBin = slice
    else:
        outBin = pd.concat([outBin, slice])

print(outBin[" Label"].value_counts())

BENIGN    56859
ATTACK    56859
Name:  Label, dtype: int64


In [32]:
multiclass.to_csv("multiclass.csv")
outBin.to_csv("binary.csv")